In [1]:
!wget -q https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

In [2]:
!curl http://localhost:9200

{
  "name" : "dd002994c245",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "_YfCdMT3RO-1ukVJNJ122w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [55]:
from elasticsearch import Elasticsearch

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


import json

import os
from dotenv import load_dotenv

from tqdm.auto import tqdm

load_dotenv()
api_key = os.getenv('api_key')

In [75]:
client = Elasticsearch('http://localhost:9200')

In [76]:
print(client.info())

{'name': 'b9363d650db0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'AlU1npKAS9KL9Zvr0F1RWA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [78]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [79]:
index_name = 'course-questions'

In [80]:
client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [81]:
with open('documents.json', 'r') as fp:
    docs_raw = json.load(fp)

In [82]:
document = []
for docs in docs_raw:
   for doc in docs['documents']:
       doc['course'] = docs['course']
       document.append(doc)

In [83]:
document[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}

In [84]:
for doc in tqdm(document):
    client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:18<00:00, 50.05it/s]


In [67]:
query = 'can i still join?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [68]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = client.search(index = index_name, body=search_query)
    result_doc = []
    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [69]:
def build_prompt(query, search_result):
    prompt_template = '''
    You're a course teaching assistant. Answer the QUESTION base on the CONTEXT from the FAQ database.
    use only the facts from the CONTEXT when answering the QUESTION.
    if the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    
    '''.strip()
    context = ''

    for doc in search_result: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [70]:
def llm(prompt):
    model = "mistral-small-latest"
    
    client = MistralClient(api_key=api_key)
    
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
    )


    return chat_response.choices[0].message.content

In [71]:
def rag(query):
    search_result = elastic_search(query)
    prompt = build_prompt(query, search_result)
    return llm(prompt)

In [72]:
print(rag('how do I get the certificate for the course?'))

To get the certificate for the course, you need to finish the course with a “live” cohort. This is because you need to peer-review capstone(s) after submitting a project, which can only be done at the time the course is running. Unfortunately, certificates are not awarded for the self-paced mode.


## query to answer the homework question

In [88]:
#Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

query = 'How do I execute a command in a running docker container?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
client.search(index=index_name, body=search_query)['hits']['max_score']

84.050095

In [95]:
# now let's only limit the questions to machine-learning-zoomcamp.

#Return 3 results. What's the 3rd question returned by the search engine?
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
response = client.search(index=index_name, body=search_query)['hits']['hits']

In [96]:
response[-1]

{'_index': 'course-questions', '_id': 'zkf0RpABbwD9Yvr-74j-', '_score': 49.938507, '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}}

In [97]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [98]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [106]:
context = ''
for doc in response:
    context = context + context_template.format(question=doc['_source']['question'], text=doc['_source']['text']) + '\n\n'

In [111]:
prompt = prompt_template.format(question=query, context=context).strip()

In [112]:
len(prompt)

1462

In [113]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.2 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 23.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [115]:
import tiktoken

In [116]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [118]:
len(encoding.encode(prompt))

322